In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import os

In [8]:
# Creates a list of strings that match the files in the directory
nums = [str(num) for num in range(1, 9)]
data = ["route-finder_" + i + ".csv" for i in nums]

In [9]:
# Creates a list of dataframes, one for each .csv file in the directory
route_finder_list = [pd.read_csv(climb) for climb in data]

In [10]:
# Stacks all of the dataframes to make one big dataframe
climbs = pd.concat(route_finder_list).reset_index()

In [10]:
# Write out the Combined version of the data so that I have a csv with all of the data in it 
#climbs.to_csv("utah_climbs.csv", index = False)

In [11]:
# This Code creates two boolean factors for whether the climb is rated PG13 or R
pg13 = climbs.Rating.str.extract("(PG13)")
R = climbs.Rating.str.extract("(R)")
climbs["PG13"] = pg13 == "PG13"
climbs["R"] = R == "R"
climbs.iloc[918]

index                                                           143
Route                                                     Altar Boy
Location          PA's Mother > Rock Canyon > Southern Wasatch >...
URL               https://www.mountainproject.com/route/10680596...
Avg Stars                                                       2.0
Your Stars                                                       -1
Route Type                                              Trad, Sport
Rating                                                 5.10b/c PG13
Pitches                                                           1
Length                                                        100.0
Area Latitude                                               40.2656
Area Longitude                                            -111.6204
PG13                                                           True
R                                                             False
Name: 918, dtype: object

In [12]:
# Remove PG13 and R from the "Rating" column
climbs["Rating"] = climbs.Rating.str.replace("(PG13|R)", "", regex = True)


In [13]:
locs = climbs.Location.apply(lambda x : x[::-1]).str.extract(r"([A-Za-z1-9 ]+)>([A-Za-z1-9 ]+)>([A-Za-z1-9 ]+)>([A-Za-z1-9 ]+)>([A-Za-z1-9 ]+)")

# for i in range(4):
#     locs[i] = locs[i].apply(lambda x : x[::-1])
locs["State"] = locs[0]
locs["Region"] = locs[1]
locs["Location"] = locs[2]
locs["Crag"] = locs[3]
locs["Wall"] = locs[4]
locs.State   #.apply(lambda x : x[::-1])

0       hatU 
1       hatU 
2       hatU 
3       hatU 
4       hatU 
        ...  
6102      NaN
6103      NaN
6104      NaN
6105      NaN
6106      NaN
Name: State, Length: 6107, dtype: object

In [14]:
climbs["Location"].str.extract("(> Utah$)").isna().sum()

0    0
dtype: int64

In [15]:
url = "https://www.mountainproject.com/gyms/utah"

In [16]:
r = requests.get(url)

In [17]:
soup = BeautifulSoup(r.text)

In [18]:
links = soup.find_all('a')

def hasGym(link):
    try:
        if "gym/" in link["href"]:
            return True
        else:
            return False
    except:
        return False

gyms = [link.text for link in links if hasGym(link)]
gyms

['Add Missing Gym',
 'AlpenFit - Indoor Training for Outdoo…',
 'Contact Climbing',
 'Dimple Dell',
 'Double-Diamond Climbing Wall',
 'Elevation Rock Gym',
 'Fit Stop Rock Climbing Gym',
 'iROCK Utah',
 'Lehi Legacy Center',
 'Momentum Indoor Climbing Gym',
 'Momentum Lehi',
 'Momentum Millcreek',
 'Salt Lake Bouldering Project',
 'SUU Climbing Gym',
 'Taylorsville Rec Center',
 'Technical Training Center',
 'The Front Climbing Club',
 'The Front Climbing Club',
 'The Front Climbing Club - South Main',
 'The Project',
 'The Quarry',
 'The Scratch Pad',
 'The Summit',
 'Uintah Recreation Community Center',
 'Ultimate Sports Arena',
 'Washington City Community Center',
 'West Valley Family Fitness Center cli…']

In [4]:
# Add in Weather Dataframe
weather = pd.read_csv('Utah_Weather_Stations.csv')
weather.head()

,X,Y,OBJECTID,STATION_NA,STN_ID_,SECONDARY_,STN_AUTHOR,NETWORK_,LOCATION_O,TYPE_OF_OB,...,SOIL_TEMP,FUEL_TEMP,FUEL_MOIST,PRESSURE,SL_PRESSUR,ALTIMETER,WEATHER_CO,VISIBILITY,UVB,ELEVATION
0,-1.257919e+07,4.439105e+06,1,CANNAN,42115T04,,NWS,COOP,http://climate.usu.edu,ONCE DAILY,...,,,,,,,,,,5000.000000
1,-1.261042e+07,4.439189e+06,2,FORT PEARCE /ST GEORGE,FPWU1,DD09129A,NOAA/NWS,HADS,http://amazon.nws.noaa.gov/hads/charts/UT.html,HOURLY,...,,,,,,,,,,
2,-1.262424e+07,4.439830e+06,3,VIRGIN RIVER/ST GEORGE 10SW,VRSU1,,1668A424,HADS,http://amazon.nws.noaa.gov/hads/charts/UT.html,HOURLY,...,,,,,,,,,,
3,-1.252687e+07,4.440500e+06,4,KANAB MUNICIPAL AIRPORT,KKNB,99999,National Weather Service,NWS/FAA,http://www.faa.gov/asos/,HOURLY,...,,,,PR,SLP,A,WC,VIS,,4863.912000
4,-1.257535e+07,4.441154e+06,5,CW2921 Hildale,C2921,9999999,APRSWXNET/Citizen Weather Observer Program,APRSWXNET/CWOP,http://www.wxqa.com/states/UT.html,HOURLY,...,,,,PR,SLP,A,,,,5155.832000


In [21]:
# Find disctinct counties (hopefully will match up with the regions from climbing)
unique_counties = weather['COUNTY'].unique()
print(unique_counties)

['WASHINGTON' 'KANE' 'SAN JUAN' 'IRON' 'GARFIELD' 'WEBER' 'DAGGETT'
 'GARFILED' 'PIUTE' 'WAYNE' 'BEAVER' 'JUAB' 'GRAND' 'SEVIER' 'MILLARD'
 'UTAH' 'DUCHESNE' 'TOOELE' 'UINTAH' 'WASATCH' 'EMERY' 'CARBON' 'SANPETE'
 'CACHE' 'SALT LAKE' 'SUMMIT' 'DAVIS' 'BOX ELDER' 'MORGAN' 'RICH']


In [19]:
for county in unique_counties:
    mask = climbs['Location'].str.contains(county)
    # Print rows where the county is found in the location column
    print(climbs[mask])

Empty DataFrame
Columns: [index, Route, Location, URL, Avg Stars, Your Stars, Route Type, Rating, Pitches, Length, Area Latitude, Area Longitude, PG13, R]
Index: []
Empty DataFrame
Columns: [index, Route, Location, URL, Avg Stars, Your Stars, Route Type, Rating, Pitches, Length, Area Latitude, Area Longitude, PG13, R]
Index: []
Empty DataFrame
Columns: [index, Route, Location, URL, Avg Stars, Your Stars, Route Type, Rating, Pitches, Length, Area Latitude, Area Longitude, PG13, R]
Index: []
Empty DataFrame
Columns: [index, Route, Location, URL, Avg Stars, Your Stars, Route Type, Rating, Pitches, Length, Area Latitude, Area Longitude, PG13, R]
Index: []
Empty DataFrame
Columns: [index, Route, Location, URL, Avg Stars, Your Stars, Route Type, Rating, Pitches, Length, Area Latitude, Area Longitude, PG13, R]
Index: []
Empty DataFrame
Columns: [index, Route, Location, URL, Avg Stars, Your Stars, Route Type, Rating, Pitches, Length, Area Latitude, Area Longitude, PG13, R]
Index: []
Empty Data